In [1]:
import CMZoom_utility as cmz
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import astrodendro
import os
from astropy import wcs
from astrodendro import Dendrogram, pp_catalog
from astrodendro.analysis import PPStatistic
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
import reproject
import pyregion
print(np.__version__)
import aplpy
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

1.16.5


In [17]:
##################################################################################################
### Start with only the output of the cataloging script, all extra columns will be added below. 
##################################################################################################
### picks for the accurate catalog
# rms = 3.0e6
# dend_min_v = 3
# dend_min_d = 1
# dend_min_pix = 17
# prune_min_peak = 6
# prune_min_mean = 2
# path = os.path.expanduser('/Users/hph/cmzoom_catalog/catalog_acc/')
# writefile = '/Users/hph/cmzoom_catalog/catalog_acc/catalog_acc.fits'
#######################################
### picks for the complete catalog
rms = 3.0e6
dend_min_v = 3
dend_min_d = 1
dend_min_pix = 17
prune_min_peak = 4
prune_min_mean = 2
path = os.path.expanduser('/Users/hph/cmzoom_catalog/catalog_complete/')
writefile = '/Users/hph/cmzoom_catalog/catalog_complete/catalog_complete.fits'
#######################################
date = '_02-02-20'
targetname = '_rms3e6_k14_dv'+str(dend_min_v)+'_dd'+str(dend_min_d)+'_dp'+str(dend_min_pix)+'_pp'+str(prune_min_peak)+'_pm'+str(prune_min_mean)+'_gal'+date

sma_path = path
herschel_path = path
catalog_path = path
figure_path = path
print(cmz.distance)

8150.0


In [18]:
### Add Herschel Column Density to table

column_file = os.path.join(herschel_path, 'column_properunits_conv36_source_only.fits')
temperature_file = os.path.join(herschel_path, 'herschel_temperature_conv36.fits')
# column density file handle
column_fh = fits.open(column_file)
temperature_fh = fits.open(temperature_file)

#temperature_file = os.path.join(herschel_path, 'temp_conv36_source_only.fits')
## temperature density file handle
#temperature_fh = fits.open(temperature_file)

#assert temperature_fh[0].data.shape == column_fh[0].data.shape


sma_file = sma_path+'CMZoom_continuum_pbcor.fits'
sma_mosaic = fits.open(sma_file)
sma_orig = sma_mosaic[0].data
sma_observed = np.isfinite(sma_orig)

smaobserved_projto_herschel,_ = reproject.reproject_interp((sma_observed,
                                                            wcs.WCS(sma_mosaic[0].header).celestial),
                                                           column_fh[0].header)
smaobserved_projto_herschel[np.isnan(smaobserved_projto_herschel)] = 0
smaobserved_projto_herschel = smaobserved_projto_herschel.astype('bool')

catalog = Table.read(catalog_path+ 'catalog_datatab'+targetname+'.fits')

colwcs = wcs.WCS(column_fh[0].header)
pix = colwcs.wcs_world2pix(catalog['glon'], catalog['glat'], 0)
column_dens = column_fh[0].data[pix[1].astype('int'), pix[0].astype('int')]

temperature = temperature_fh[0].data[pix[1].astype('int'), pix[0].astype('int')]
print(catalog['glon'][np.where(np.isnan(temperature))[0]])
print(catalog['glon'][np.where(np.isnan(column_dens))[0]])
catalog.add_column(Column(name='DustTemperature', data=temperature))
catalog['DustTemperature'][np.isnan(catalog['DustTemperature'])]=20.0
catalog.add_column(Column(name='herschel_column', data=column_dens))
#catalog.write(catalog_path + 'datab_with_ColumnDensity'+targetname+'.fits', overwrite=True)

/Users/hph/anaconda/envs/cmzoom3/lib/python3.6/site-packages/reproject/interpolation/core_celestial.py:26: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(array.dtype, np.float):


       glon       
       deg        
------------------
0.6668037580754469
0.6648988374066647
0.6614362030011472
0.6664111934458868
0.6644910834144118
0.6653422193647489
0.6723806363724764
0.6776647061125441
0.6779974038947394
0.6732040152820338
       glon       
       deg        
------------------
0.6668037580754469
0.6648988374066647
0.6614362030011472
0.6664111934458868
0.6644910834144118
0.6653422193647489
0.6723806363724764
0.6776647061125441
0.6779974038947394
0.6732040152820338


In [19]:
# Generate a master table to store all of the information
table = Table(meta={'name': 'Leaf Properties'})
table_master = Table(meta={'name': 'Master Properties'})
headings_master = ['mass', 'sma_column','r_eff_pc', 'Sigma', 'n', 'rho', 'tff','Sigma','glon_0','mass_bgsub']
description_master = ['g2d=100, beta=1.75','', 'beta=1.75, mu=2.8', '','', '', '', '','','deg', '']
# headings = ['mass', 'N', 'r_eff_pc', 'Sigma', 'n', 'rho', 'tff', 'mass_bgsub']
# description = ['g2d=100, beta=1.75', 'beta=1.75, mu=2.8', '', '', '', '', '','']

# Update the table with Herschel-derived temperature
#dustTemp = cmz.dustTemp

# physical Properties
r_eff_pc = catalog['r_eff'] * cmz.arcsec2pc
mass = cmz.mass_calc_submm(cmz.Wave, catalog['DustTemperature'], cmz.Kappa, catalog['flux_integrated'], cmz.distance)
N = cmz.column_density(cmz.Wave, catalog['DustTemperature'], cmz.Kappa, catalog['peak_cont_flux'].data, cmz.mu)
Sigma = N * cmz.mh * 1000.0
n = cmz.number_density_sphere_pc(mass, catalog['r_eff'].data*cmz.arcsec2pc, cmz.mu)
rho = cmz.mass_density_sphere(mass, catalog['r_eff'].data*cmz.arcsec2pc)
tff = cmz.tff_spherical_F(n, cmz.mu)
mass_bgsub = cmz.mass_calc_submm(cmz.Wave, catalog['DustTemperature'], cmz.Kappa, catalog['flux_integrated_bgsub'].data, cmz.distance)
minflux_integrated = catalog['min_cont_flux'].data*cmz.JyperSr_to_Jyperassqr*catalog['area_exact'].data
flux_integrated_bgsub_as = catalog['flux_integrated'].data - minflux_integrated
mass_bgsub_pix = cmz.mass_calc_submm(cmz.Wave, catalog['DustTemperature'], cmz.Kappa, catalog['flux_integrated_bgsub'].data, cmz.distance)

mass = mass
print("Total mass in leaf structures is " + str(sum(mass)) )
# n.unit = u.cm**-3
# N.unit =  u.cm**-2
# Sigma = Sigma * u.g * u.cm**-2
# rho.unit = u.kg * u.m**-3
# tff.unit = u.yr
glon_0 = catalog['glon']
for i in range(len(glon_0)):
    if catalog['glon'][i] > 5:
        glon_0[i] = glon_0[i] - 360.
glon_0 = -glon_0
mass_bgsub = mass_bgsub*u.Msun

# properties = [mass, N, r_eff_pc, Sigma, n, rho, tff, glon_0,mass_bgsub]

# for i in range(len(headings)):
#     table[headings[i]] = Column(properties[i], description=description[i])

properties_master = [mass, N, r_eff_pc, Sigma, n, rho, tff,Sigma,glon_0,mass_bgsub]

for i in range(len(headings_master)):
    table_master[headings_master[i]] = Column(properties_master[i], description=description_master[i])

catalog = hstack([catalog, table_master])

catalog['n'].unit=u.cm**-3
catalog['sma_column'].unit=u.cm**-2
catalog['herschel_column'].unit=u.cm**-2
catalog['rho'].unit = u.kg * u.m**-3
catalog['tff'].unit = u.yr
catalog['Sigma'].unit= u.g * u.cm**-2

Total mass in leaf structures is 243492.43982168922


In [20]:
###########################################
### flag leaves close to edge in new column
###     (if you want to...)
###########################################

directory = '/Users/hph/cmzoom_catalog/catalog_acc/'
sma = fits.open(directory + 'CMZoom_continuum_pbcor.fits')
image = sma[0].data
sma_wcs = wcs.WCS(sma[0])

susp_r = 10
suspicious = np.zeros(catalog['glon'].shape)
print(suspicious.shape)
ycoords, xcoords = np.ogrid[:(image.shape[0]), :(image.shape[1])]
for s in range(len(suspicious)):
    cglon, cglat = catalog['glon'][s],catalog['glat'][s]
    cx, cy = sma_wcs.wcs_world2pix(cglon,cglat,1)[0], sma_wcs.wcs_world2pix(cglon,cglat,1)[1]
    region = np.where( susp_r**2 >(cx-xcoords)**2 + (cy-ycoords)**2 )
    if np.any(np.isnan(image[region])): suspicious[s]=True
    print(s/float(len(suspicious)), suspicious[s])
    
###########################################
### save suspicious column in catalog
###########################################
#catalog.remove_column('suspicious')
catalog.add_column(Column(name='suspicious', data=suspicious))


(816,)
0.0 0.0
0.0012254901960784314 0.0
0.0024509803921568627 0.0
0.003676470588235294 0.0
0.004901960784313725 0.0
0.006127450980392157 0.0
0.007352941176470588 0.0
0.00857843137254902 0.0
0.00980392156862745 1.0
0.011029411764705883 0.0
0.012254901960784314 0.0
0.013480392156862746 0.0
0.014705882352941176 0.0
0.015931372549019607 0.0
0.01715686274509804 0.0
0.01838235294117647 0.0
0.0196078431372549 0.0
0.020833333333333332 0.0
0.022058823529411766 0.0
0.023284313725490197 0.0
0.024509803921568627 0.0
0.025735294117647058 0.0
0.02696078431372549 0.0
0.028186274509803922 0.0
0.029411764705882353 0.0
0.030637254901960783 0.0
0.031862745098039214 1.0
0.03308823529411765 0.0
0.03431372549019608 1.0
0.03553921568627451 0.0
0.03676470588235294 1.0
0.03799019607843137 0.0
0.0392156862745098 1.0
0.04044117647058824 0.0
0.041666666666666664 0.0
0.0428921568627451 0.0
0.04411764705882353 0.0
0.04534313725490196 0.0
0.04656862745098039 0.0
0.04779411764705882 0.0
0.049019607843137254 0.0
0.05

0.4264705882352941 1.0
0.42769607843137253 0.0
0.42892156862745096 0.0
0.43014705882352944 0.0
0.43137254901960786 0.0
0.4325980392156863 0.0
0.4338235294117647 0.0
0.43504901960784315 0.0
0.4362745098039216 0.0
0.4375 0.0
0.4387254901960784 0.0
0.43995098039215685 0.0
0.4411764705882353 0.0
0.4424019607843137 0.0
0.44362745098039214 0.0
0.44485294117647056 0.0
0.44607843137254904 0.0
0.44730392156862747 0.0
0.4485294117647059 0.0
0.4497549019607843 0.0
0.45098039215686275 0.0
0.4522058823529412 0.0
0.4534313725490196 0.0
0.45465686274509803 0.0
0.45588235294117646 0.0
0.4571078431372549 0.0
0.4583333333333333 0.0
0.45955882352941174 0.0
0.46078431372549017 0.0
0.46200980392156865 0.0
0.4632352941176471 0.0
0.4644607843137255 0.0
0.46568627450980393 0.0
0.46691176470588236 0.0
0.4681372549019608 0.0
0.4693627450980392 0.0
0.47058823529411764 1.0
0.47181372549019607 0.0
0.4730392156862745 0.0
0.4742647058823529 1.0
0.47549019607843135 0.0
0.47671568627450983 0.0
0.47794117647058826 0.0


0.8676470588235294 0.0
0.8688725490196079 0.0
0.8700980392156863 0.0
0.8713235294117647 0.0
0.8725490196078431 0.0
0.8737745098039216 0.0
0.875 0.0
0.8762254901960784 0.0
0.8774509803921569 0.0
0.8786764705882353 0.0
0.8799019607843137 0.0
0.8811274509803921 0.0
0.8823529411764706 0.0
0.883578431372549 0.0
0.8848039215686274 0.0
0.8860294117647058 0.0
0.8872549019607843 0.0
0.8884803921568627 1.0
0.8897058823529411 0.0
0.8909313725490197 0.0
0.8921568627450981 0.0
0.8933823529411765 0.0
0.8946078431372549 0.0
0.8958333333333334 1.0
0.8970588235294118 1.0
0.8982843137254902 0.0
0.8995098039215687 1.0
0.9007352941176471 0.0
0.9019607843137255 0.0
0.9031862745098039 0.0
0.9044117647058824 0.0
0.9056372549019608 0.0
0.9068627450980392 0.0
0.9080882352941176 1.0
0.9093137254901961 0.0
0.9105392156862745 0.0
0.9117647058823529 0.0
0.9129901960784313 0.0
0.9142156862745098 0.0
0.9154411764705882 0.0
0.9166666666666666 0.0
0.9178921568627451 0.0
0.9191176470588235 0.0
0.9203431372549019 0.0
0.

In [13]:
#catalog.add_column(Column(name='suspicious', data=suspicious))

In [21]:
print(len(suspicious))
print(np.where(suspicious==1)[0])

816
[  8  26  28  30  32  41  60  80  88  93 103 108 115 119 125 144 146 148
 189 198 217 252 298 348 384 387 396 430 431 540 550 551 594 603 604 611
 613 615 656 669 671 682 685 695 725 731 732 734 741 754 769 772 785 811
 812 814 815]


In [22]:
##################################################
### reindex catalog
### alphabetical for each cloud
### in order of flux (max --> min)
##################################################
region_mask = fits.open('/Users/hph/cmzoom_catalog/Continuum/CMZoom_Mask_new.fits')
mask_data = region_mask[0].data
catalog.add_column(np.zeros_like(catalog['mass'],dtype='str'), name='leaf_ID')

In [23]:
##################################################
### define regions for reindexing
##################################################
names = np.array(['G1.683-0.089', 
                      'G1.670-0.130', 
                      'G1.651-0.050', 
                      'G1.602+0.018',
                      'G1.085-0.027', 
                      'G1.038-0.074', 
                      'G0.891-0.048',
                      'G0.714-0.100',
                      'G0.699-0.028',
                      'G0.619+0.012',
                      'G0.489+0.010',
                      'G0.412+0.052',
                      'G0.393-0.034',
                      'G0.380+0.050',
                      'G0.340+0.055',
                      'G0.326-0.085',
                      'G0.316-0.201',
                      'G0.253+0.016',
                      'G0.212-0.001',
                      'G0.145-0.086',
                      'G0.106-0.082',
                      'G0.070-0.035',
                      'G0.068-0.075',
                      'G0.054+0.027',
                      'G0.014+0.021',
                      'G0.001-0.058',
                      'G359.948-0.052',
                      'G359.889-0.093',
                      'G359.865+0.022',
                      'G359.863-0.069',
                      'G359.734+0.002',
                      'G359.648-0.133',
                      'G359.611+0.018',
                      'G359.615-0.243',
                      'G359.484-0.132',
                      'G359.137+0.031'])
#'G1.127-0.106', <--- who is this?? trouble region...

In [16]:
print(catalog['glon'][734],catalog['glat'][734])
print(cloud_id[734])

IndexError: index 734 is out of bounds for axis 0 with size 285

In [24]:
##################################################
### reindexing continued...
##################################################
w = wcs.WCS(path+'CMZoom_continuum_pbcor.fits').celestial

xglon = w.wcs_world2pix(catalog['glon'],catalog['glat'],0)[0]
yglat = w.wcs_world2pix(catalog['glon'],catalog['glat'],0)[1]

### define alphabet convention (seriously must be a better way...)
abc = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
       'o','p','q','r','s','t','u','v','w','x','y','z','aa','ab',
       'ac','ad','ae','af','ag','ah','ai','aj','ak','al','am','an','ao','ap',
       'aq','ar','as','at','au','av','aw','ax','ay','az','ba','bb',
       'bc','bd','be','bf','bg','bh','bi','bj','bk','bl','bm','bn','bo','bp',
       'bq','br','bs','bt','bu','bv','bw','bx','by','bz''ca','cb',
       'cc','cd','ce','cf','cg','ch','ci','cj','ck','cl','cm','cn','co','cp',
       'cq','cr','cs','ct','cu','cv','cw','cx','cy','cz']

problem_list=np.empty(0)
cloud_id = np.empty(len(catalog['mass']),dtype=object)
mask_num = cloud_id.copy()
for i in range(len(cloud_id)):
    cnum = int(mask_data[int(yglat[i])][int(xglon[i])])
    mask_num[i]=cnum
    if(cnum==1000):
          np.append(problem_list,i)
    else:    
        cloud_id[i]=str(names[int(mask_data[int(yglat[i])][int(xglon[i])])-1])
### handle special cases: WARNING, THIS MUST BE DONE BY EYE for now...
# cloud_id[36]=names[0]   ### trouble in 1.7 degree cloud for acc catalog

cloud_id[119]=names[0]   ### trouble for complete catalog
cloud_id[125]=names[0]   ### trouble for complete catalog
cloud_id[144]=names[15]   ### trouble for complete catalog
cloud_id[198]=names[15]   ### trouble for complete catalog
cloud_id[252]=names[2]   ### trouble for complete catalog
cloud_id[615]=names[29]   ### trouble for complete catalog
cloud_id[656]=names[28]   ### trouble for complete catalog
cloud_id[734]=names[13]   ### trouble for complete catalog



#print(cloud_id)

leaf_ID = cloud_id.copy()
### append mass ordered label
total=0
for i in range(len(cloud_id)):
    cmasses=np.flip(np.sort(catalog['mass'][np.where(cloud_id==cloud_id[i])]))
    #print(len(cmasses))
    #print(np.where(cmasses==catalog['mass'][i])[0][0])
    total = total+np.where(cmasses==catalog['mass'][i])[0][0]
    print(i)
    leaf_ID[i]=cloud_id[i]+abc[np.where(cmasses==catalog['mass'][i])[0][0]]
    
catalog['leaf_ID']=leaf_ID.copy().astype(str)
catalog['mask_num']=mask_num.copy().astype(int)



    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [61]:
print(catalog['mass'][i])

8.912972022654667


In [25]:
########################################################
### remove temporary columns
########################################################
catalog.remove_columns(['is_leaf','testflux_integrated'])


In [26]:
##########################################################
### create unc columns
##########################################################
def mass_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa):
    '''
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    '''
    d = d * cmz.pc2cm
    d_err = d_err * cmz.pc2cm
    I_int_err = I_peak_err*(area/(3600*3600*3282.8)) #convert area to sr 
    B = cmz.planck_wave(cmz.Wave,Td)
    dterm = (2*d*I_int*d_err)/(cmz.Kappa*B)
    Iterm = (I_int_err*d**2)/(cmz.Kappa*B)
    Tterm = Td_err * ((I_int * d**2)/cmz.Kappa) * (((1.0e-26)*(cmz.Wave**2))/(2*cmz.kboltzmann*(Td**2))) *np.exp(cmz.hplanck*cmz.clight/(cmz.kboltzmann*cmz.Wave*Td))
    return np.sqrt(dterm**2 + Iterm**2 + Tterm**2)/cmz.msun


def N_err(I_peak, I_peak_err, Td, Td_err, Kappa):
    '''
    I_peak: peak flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    Kappa: assumed dust opacity in cm^2 /gram
    '''
    B = cmz.planck_wave(cmz.Wave,Td)
    Iterm = (I_peak_err)/(cmz.mu * (cmz.mh*1.e3) * cmz.Kappa * B)
    Tterm = Td_err * (I_peak/(cmz.mu * (cmz.mh*1.e3) * cmz.Kappa)) * (((1.0e-26)*(cmz.Wave**2))/(2*cmz.kboltzmann*(Td**2))) *np.exp(cmz.hplanck*cmz.clight/(cmz.kboltzmann*cmz.Wave*Td))
    return np.sqrt(Iterm**2 + Tterm**2)

def ndens_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    """
    Radius = Radius_pc * (cmz.pc2cm/100.0) #convert to SI
    m_err = mass_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                     Td=20.0,Td_err=5.0,
                     d=8150.0,d_err=150.0,
                     area=catalog['area_exact'],Kappa=cmz.Kappa)
    Mass = m_err*cmz.msun / (1000.0) #convert to SI
    n_err = Mass/(((4./3.)*np.pi) * cmz.mu * cmz.mh * Radius**3.0)
    n_err = n_err / cmz.percm2perm
    return n_err

def rho_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    returns mass density in 
    """
    Radius = Radius_pc * (cmz.pc2cm/100.0) #convert to SI
    m_err = mass_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                     Td=20.0,Td_err=5.0,
                     d=8150.0,d_err=150.0,
                     area=catalog['area_exact'],Kappa=Kappa)
    Mass = m_err*cmz.msun / (1000.0) #convert to cgs
    rho_err = Mass/(((4./3.)*np.pi) * Radius**3.0)

    return rho_err
    
def tff_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc, mass_density):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    returns freefall time error in years
    """
    number_density_err = ndens_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc)
    mass_density_err = cmz.mu * cmz.mh * number_density_err * cmz.percm2perm
    sigma_tff = np.sqrt((3. * np.pi) / (32. * cmz.G * mass_density**3))*mass_density_err
    sigma_tff = sigma_tff / cmz.sin1yr # free-fall time in years

    return sigma_tff

sigma_mass = mass_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                      Td=catalog['DustTemperature'],Td_err=5.0,
                      d=8150.0,d_err=150.0,
                      area=catalog['area_exact'],Kappa=cmz.Kappa)
sigma_ndens = ndens_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                      Td=catalog['DustTemperature'],Td_err=5.0,
                      d=8150.0,d_err=150.0,
                      area=catalog['area_exact'],Kappa=cmz.Kappa,Radius_pc=catalog['r_eff_pc'])
sigma_rho = rho_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                      Td=catalog['DustTemperature'],Td_err=5.0,
                      d=8150.0,d_err=150.0,
                      area=catalog['area_exact'],Kappa=cmz.Kappa,Radius_pc=catalog['r_eff_pc'])
sigma_column = N_err(I_peak=catalog['peak_cont_flux'],I_peak_err=catalog['noise'],
                     Td=catalog['DustTemperature'],Td_err=5.0,Kappa=cmz.Kappa)
sigma_tff = tff_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                    Td=catalog['DustTemperature'],Td_err=5.0,
                    d=8150.0,d_err=150.0,
                    area=catalog['area_exact'],Kappa=cmz.Kappa,Radius_pc=catalog['r_eff_pc'],
                    mass_density=catalog['rho'])
####################
# add unc columns
####################
path = '/Users/hph/cmzoom_catalog/catalog_complete/'

#catalog = Table.read(path+'catalog_complete.fits')
catalog.add_columns(cols=[sigma_mass,sigma_column,sigma_ndens,sigma_rho,sigma_tff],
                    names=['mass_unc','N_unc','n_unc','rho_unc','tff_unc'])
catalog['mass_unc'].unit=u.solMass
catalog['N_unc'].unit = u.cm**(-2)
catalog['n_unc'].unit = 1./u.cm**3
catalog['rho_unc'].unit = u.kg/u.m**3
catalog['tff_unc'].unit = u.s

In [27]:
print("writing %s" %\
      (writefile))
catalog.write(writefile, format='fits', overwrite=True)

writing /Users/hph/cmzoom_catalog/catalog_complete/catalog_complete.fits


In [4]:
catalog = Table.read('/Users/hph/cmzoom_catalog/catalog_acc/catalog_acc.fits')

In [28]:
###################################################
### make LaTeX table
###################################################

print('\\begin{tabular}{lllllllll}')
print('\hline\hline')
print('Leaf ID & Area & $l$ & $b$ & R_{eff} & Integrated Flux & Peak Cont. Flux & Mean Cont. Flux & RMS')
print(' & as$^2$ & deg. & deg. & pc & Jy & Jy/Sr & Jy/Sr & Jy/Sr')              
print('\hline')

for i in np.arange(0,10):
    print('%s & %1.2f & %1.2f & %1.2f & %1.2f & %1.2e & %1.2e & %1.2e & %1.2e' %\
         (catalog['leaf_ID'][i],catalog['area_exact'][i],catalog['glon'][i],catalog['glat'][i],catalog['r_eff_pc'][i],catalog['flux_integrated'][i],catalog['peak_cont_flux'][i],catalog['mean_cont_flux'][i],catalog['noise'][i]))
        
print('\hline\hline')
print('\end{tabular}')
print(' ')
print(' ')  

\begin{tabular}{lllllllll}
\hline\hline
Leaf ID & Area & $l$ & $b$ & R_{eff} & Integrated Flux & Peak Cont. Flux & Mean Cont. Flux & RMS
 & as$^2$ & deg. & deg. & pc & Jy & Jy/Sr & Jy/Sr & Jy/Sr
\hline
G359.611+0.018c & 45.00 & -0.38 & -0.25 & 0.15 & 6.71e-02 & 1.39e+08 & 6.35e+07 & 1.68e+07
G359.611+0.018g & 14.00 & -0.38 & -0.25 & 0.08 & 1.64e-02 & 6.51e+07 & 4.97e+07 & 1.53e+07
G359.611+0.018d & 43.75 & -0.39 & -0.25 & 0.15 & 3.35e-02 & 7.06e+07 & 3.26e+07 & 1.44e+07
G359.611+0.018a & 259.25 & -0.39 & -0.24 & 0.36 & 2.09e+00 & 1.73e+09 & 3.43e+08 & 9.88e+06
G359.611+0.018i & 4.75 & -0.38 & -0.25 & 0.05 & 6.50e-03 & 6.35e+07 & 5.82e+07 & 1.17e+07
G359.611+0.018b & 34.75 & -0.38 & -0.24 & 0.13 & 7.52e-02 & 1.70e+08 & 9.20e+07 & 1.25e+07
G359.611+0.018h & 15.50 & -0.39 & -0.24 & 0.09 & 1.43e-02 & 5.91e+07 & 3.92e+07 & 1.30e+07
G359.611+0.018f & 6.75 & -0.40 & -0.24 & 0.06 & 1.95e-02 & 1.48e+08 & 1.23e+08 & 3.13e+07
G359.611+0.018e & 8.00 & -0.40 & -0.24 & 0.06 & 2.24e-02 & 1.40e+08 & 1

In [29]:

print('\\begin{tabular}{llllllll}')
print('\hline\hline')
print('Leaf ID & N$_{Herschel}$ & N$_{SMA}$ & Mass & Mass (bg. sub.) & n & $\\rho$ & t$_{ff}$ \\')
print(' & cm$^{-2}$ & cm$^{-2} & M$_\odot$ & M$_\odot$ & cm$^{-3}$ & g cm$^{-3}$ & years \\')              
print('\hline')

for i in np.arange(0,10):
    print('%s & %1.2e & %1.2e& %1.2e & %1.2e & %1.2e& %1.2e & %1.2e \\' %\
         (catalog['leaf_ID'][i],catalog['herschel_column'][i],catalog['sma_column'][i],catalog['mass'][i],catalog['mass_bgsub'][i],catalog['n'][i],catalog['rho'][i],catalog['tff'][i]))
        
print('\hline\hline')
print('\end{tabular}')


\begin{tabular}{llllllll}
\hline\hline
Leaf ID & N$_{Herschel}$ & N$_{SMA}$ & Mass & Mass (bg. sub.) & n & $\rho$ & t$_{ff}$ \
 & cm$^{-2}$ & cm$^{-2} & M$_\odot$ & M$_\odot$ & cm$^{-3}$ & g cm$^{-3}$ & years \
\hline
G359.611+0.018c & 6.81e+22 & 1.42e+23& 1.02e+02 & 4.13e+01 & 1.05e+05& 4.94e-16 & 9.47e+04 \
G359.611+0.018g & 6.81e+22 & 6.65e+22& 2.49e+01 & 5.90e+00 & 1.48e+05& 6.94e-16 & 7.99e+04 \
G359.611+0.018d & 8.04e+22 & 6.36e+22& 4.50e+01 & 2.35e+01 & 4.84e+04& 2.27e-16 & 1.40e+05 \
G359.611+0.018a & 1.46e+23 & 1.73e+24& 3.13e+03 & 2.61e+03 & 2.34e+05& 1.09e-15 & 6.36e+04 \
G359.611+0.018i & 9.96e+22 & 6.20e+22& 9.45e+00 & 8.39e-01 & 2.84e+05& 1.33e-15 & 5.77e+04 \
G359.611+0.018b & 8.59e+22 & 1.73e+23& 1.14e+02 & 4.25e+01 & 1.73e+05& 8.09e-16 & 7.40e+04 \
G359.611+0.018h & 1.31e+23 & 5.57e+22& 2.01e+01 & 7.47e+00 & 1.03e+05& 4.80e-16 & 9.60e+04 \
G359.611+0.018f & 5.33e+22 & 1.43e+23& 2.81e+01 & 4.73e+00 & 5.00e+05& 2.34e-15 & 4.35e+04 \
G359.611+0.018e & 5.33e+22 & 1.36e+23&

In [30]:

print('\\begin{tabular}{llllll}')
print('\hline\hline')
print('Leaf ID & Mass Unc. & N$_{SMA}$ unc. & n unc. & $\\rho$ unc. & t$_{ff}$ unc. \\')
print(' & M$_\odot$ & cm$^{-2}$ & cm$^{-3}$ & g cm$^{-3}$ years \\')              
print('\hline')

for i in np.arange(0,10):
    print('%s & %1.2e & %1.2e& %1.2e & %1.2e & %1.2e \\' %\
         (catalog['leaf_ID'][i],catalog['mass_unc'][i],catalog['N_unc'][i],catalog['n_unc'][i],catalog['rho_unc'][i],catalog['tff_unc'][i]))
        
print('\hline\hline')
print('\end{tabular}')


\begin{tabular}{llllll}
\hline\hline
Leaf ID & Mass Unc. & N$_{SMA}$ unc. & n unc. & $\rho$ unc. & t$_{ff}$ unc. \
 & M$_\odot$ & cm$^{-2}$ & cm$^{-3}$ & g cm$^{-3}$ years \
\hline
G359.611+0.018c & 4.25e+01 & 4.82e+22& 4.52e+04 & 2.12e-16 & 4.06e+04 \
G359.611+0.018g & 1.10e+01 & 2.63e+22& 6.76e+04 & 3.17e-16 & 3.65e+04 \
G359.611+0.018d & 2.36e+01 & 2.21e+22& 3.07e+04 & 1.44e-16 & 8.87e+04 \
G359.611+0.018a & 9.90e+02 & 5.42e+23& 7.93e+04 & 3.72e-16 & 2.16e+04 \
G359.611+0.018i & 3.46e+00 & 2.20e+22& 1.16e+05 & 5.45e-16 & 2.36e+04 \
G359.611+0.018b & 3.92e+01 & 5.60e+22& 6.26e+04 & 2.93e-16 & 2.68e+04 \
G359.611+0.018h & 8.93e+00 & 2.05e+22& 5.26e+04 & 2.47e-16 & 4.93e+04 \
G359.611+0.018f & 1.11e+01 & 5.27e+22& 2.23e+05 & 1.04e-15 & 1.94e+04 \
G359.611+0.018e & 1.35e+01 & 5.26e+22& 2.08e+05 & 9.73e-16 & 2.16e+04 \
G0.316-0.201f & 3.30e+01 & 4.50e+22& 7.33e+04 & 3.43e-16 & 3.80e+04 \
\hline\hline
\end{tabular}


In [5]:
region_names = np.array(['G1.683-0.089', 
                      'G1.670-0.130', 
                      'G1.651-0.050', 
                      'G1.602+0.018',
                      'G1.085-0.027', 
                      'G1.038-0.074', 
                      'G0.891-0.048',
                      'G0.714-0.100',
                      'G0.699-0.028',
                      'G0.619+0.012',
                      'G0.489+0.010',
                      'G0.412+0.052',
                      'G0.393-0.034',
                      'G0.380+0.050',
                      'G0.340+0.055',
                      'G0.326-0.085',
                      'G0.316-0.201',
                      'G0.253+0.016',
                      'G0.212-0.001',
                      'G0.145-0.086',
                      'G0.106-0.082',
                      'G0.070-0.035',
                      'G0.068-0.075',
                      'G0.054+0.027',
                      'G0.014+0.021',
                      'G0.001-0.058',
                      'G359.948-0.052',
                      'G359.889-0.093',
                      'G359.865+0.022',
                      'G359.863-0.069',
                      'G359.734+0.002',
                      'G359.648-0.133',
                      'G359.611+0.018',
                      'G359.615-0.243',
                      'G359.484-0.132',
                      'G359.137+0.031'])
print(len(region_names))

36


In [8]:
catalog = Table.read('/Users/hph/cmzoom_catalog/catalog_complete/catalog_complete.fits')

In [2]:
catalog = Table.read('/Users/hph/cmzoom_catalog/catalog_acc/catalog_acc.fits')

In [3]:
select = np.where(catalog['mask_num']==26)[0]
print(select)
print('Leaf ID \t GLON \t GLAT \t SMA N \t Herschel N \t SMA Mass')
for i in range(len(select)):
    print('%s \t %0.4f \t %0.4f \t %1.2e \t %1.2e \t %1.2e' %\
          (catalog['leaf_ID'][select[i]], catalog['glon'][select[i]], catalog['glat'][select[i]], catalog['sma_column'][select[i]], catalog['herschel_column'][select[i]], catalog['mass'][select[i]]))
                          
                          
                          

[ 51  72  75  78  79  80  83  85  86  89  90  91  93  96 100 101 103 110
 131]
Leaf ID 	 GLON 	 GLAT 	 SMA N 	 Herschel N 	 SMA Mass
G0.001-0.058k 	 -0.0194 	 -0.0883 	 5.19e+22 	 5.23e+22 	 3.33e+01
G0.001-0.058p 	 -0.0114 	 -0.0830 	 5.39e+22 	 1.01e+23 	 1.77e+01
G0.001-0.058m 	 -0.0173 	 -0.0822 	 4.64e+22 	 9.38e+22 	 2.23e+01
G0.001-0.058n 	 -0.0116 	 -0.0812 	 5.38e+22 	 1.01e+23 	 1.95e+01
G0.001-0.058d 	 -0.0253 	 -0.0811 	 7.36e+22 	 7.20e+22 	 6.71e+01
G0.001-0.058h 	 -0.0326 	 -0.0815 	 1.40e+23 	 7.13e+22 	 5.08e+01
G0.001-0.058q 	 -0.0091 	 -0.0807 	 6.98e+22 	 1.17e+23 	 1.35e+01
G0.001-0.058l 	 -0.0134 	 -0.0791 	 6.15e+22 	 1.16e+23 	 3.26e+01
G0.001-0.058s 	 -0.0094 	 -0.0795 	 6.05e+22 	 1.17e+23 	 1.16e+01
G0.001-0.058c 	 -0.0380 	 -0.0791 	 7.23e+22 	 1.02e+23 	 6.86e+01
G0.001-0.058a 	 -0.0308 	 -0.0792 	 1.24e+23 	 1.04e+23 	 1.53e+02
G0.001-0.058r 	 -0.0287 	 -0.0795 	 7.23e+22 	 9.74e+22 	 1.22e+01
G0.001-0.058o 	 -0.0218 	 -0.0776 	 5.83e+22 	 1.16e+23 	 1.93e

In [9]:
########################################################
### generate txt table for every cloud with >10 leaves
########################################################
#path='/Users/hph/cmzoom_catalog/catalog_acc/'
path='/Users/hph/cmzoom_catalog/catalog_complete/'
for j in range(36):
    if len(np.where(catalog['mask_num']==j+1)[0])>10:
        select = np.where(catalog['mask_num']==j+1)[0]
        print(region_names[j])
        #file=open(path+region_names[j]+'robust.txt','w')
        file=open(path+region_names[j]+'complete.txt','w')        
        file.write('Leaf ID \t GLON \t GLAT \t SMA N \t Herschel N \t SMA Mass \n')
        for i in range(len(select)):
            file.write('%s \t %0.4f \t %0.4f \t %1.2e \t %1.2e \t %1.2e \n' %\
              (catalog['leaf_ID'][select[i]], catalog['glon'][select[i]], catalog['glat'][select[i]], catalog['sma_column'][select[i]], catalog['herschel_column'][select[i]], catalog['mass'][select[i]]))
        file.close()


G1.670-0.130
G1.651-0.050
G1.602+0.018
G1.085-0.027
G0.714-0.100
G0.699-0.028
G0.619+0.012
G0.489+0.010
G0.412+0.052
G0.380+0.050
G0.340+0.055
G0.326-0.085
G0.316-0.201
G0.253+0.016
G0.145-0.086
G0.106-0.082
G0.070-0.035
G0.068-0.075
G0.001-0.058
G359.948-0.052
G359.889-0.093
G359.734+0.002
G359.615-0.243
G359.484-0.132


In [15]:
catalog['rho']

4.940020358736823e-16
1.0945778532435116e-15
8.093304114129252e-16
6.229282868101701e-16
9.828769785209523e-16
9.060881251575829e-16
1.3270754170146477e-15
1.0316083233374472e-15
2.05061879205277e-15
1.2514874764653508e-15
5.658428447763987e-16
